In [1]:
import glob
import pandas as pd
import random

random.seed(42)
path = '../data/train_code'

In [2]:
pair_proportion = 0.5 #레이블 비율 설정(같은 문제를 해결하는 코드끼리 묶여있는 짝의 비율)

pair = int(250*pair_proportion)
not_pair = 250-pair
sample_num = random.sample([i for i in range(500)], k=int(pair*2))
similar = [True if i in sample_num else False for i in range(500)]
not_similar = [False if i in sample_num else True for i in range(500)]

df = pd.DataFrame()
df2 = pd.DataFrame()

In [3]:
for idx in range(1,501):
    code_paths = glob.glob(path+'/problem{idx:03d}/*.cpp'.format(idx=idx))
    codes = [''.join(open('{code_path}'.format(code_path=code_path),encoding='utf-8').readlines()) for code_path in code_paths]
    df = pd.concat([df, pd.DataFrame({'problem{idx:03d}'.format(idx=idx): codes})], axis=1)

In [4]:
temp = df[similar]
for idx in range(1,501):
    temp1 = list(temp['problem{idx:03d}'.format(idx=idx)])
    code1 = [temp1[i] for i in range(0,pair*2,2)]
    code2 = [temp1[i+1] for i in range(0,pair*2,2)]
    similar = [1 for _ in range(0,pair*2,2)]
    df2 = pd.concat([df2,pd.DataFrame({'code1': code1, 'code2': code2, 'similar': similar})])

In [15]:
df2

,code1,code2,similar
0,"#include ""iostream""\n#include ""climits""\n#incl...",#include <iostream>\n#include <map>\n\n#define...,1
1,// E - Sequence Sum\n#include <bits/stdc++.h>\...,//////////////////////// W H I T E C L O U ...,1
2,"#include <bits/stdc++.h>\n#define rep(i, n) fo...",#include <bits/stdc++.h>\nusing namespace std;...,1
3,#include <bits/stdc++.h>\nusing namespace std;...,#include <iostream>\n#include <unordered_map>\...,1
4,#include<bits/stdc++.h>\nusing namespace std;\...,#include <bits/stdc++.h>\n\nusing namespace st...,1
...,...,...,...
62495,#include<iostream>\nusing namespace std;\nint ...,#include <iostream>\nusing namespace std;\nint...,1
62496,#include <bits/stdc++.h>\nusing namespace std;...,"#include <bits/stdc++.h>\n#define REP(i, e) fo...",1
62497,"#include<bits/stdc++.h>\n#define rep(i,n)for(i...",#include<bits/stdc++.h>\n#define fast ios::syn...,1
62498,#include <bits/stdc++.h>\nusing namespace std;...,#include <bits/stdc++.h>\n\nusing namespace st...,1


In [6]:
df2.isna().sum()

code1      0
code2      0
similar    0
dtype: int64

In [7]:
df2 = df2.sample(frac=1).reset_index(drop=True)

In [8]:
df2.to_csv('../data/train_generator.csv', index=False)

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import random
import re

from tqdm import tqdm
from itertools import combinations
from collections import deque
from transformers import AutoTokenizer
# from transformers import AutoModel, AutoModelForSequenceClassification
# from datasets import load_dataset, load_metric
from rank_bm25 import BM25Okapi
from sklearn.model_selection import train_test_split

In [2]:
import re
from collections import deque

# C++ 코드 전처리를 위한 함수
def preprocess_script(script_path):
    new_script = deque()
    with open(script_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        block_comment = False
        for line in lines:
            # 블록 주석 시작
            if '/*' in line:
                block_comment = True
                line = line[:line.index('/*')]
            # 블록 주석 끝
            if '*/' in line:
                block_comment = False
                line = line[line.index('*/')+2:]
            if block_comment or line.strip().startswith('//'):
                continue

            line = line.rstrip()
            # 한 줄 주석 처리
            if '//' in line:
                line = line[:line.index('//')]
            line = line.replace('\n', '')  # 개행 문자를 모두 삭제함
            line = line.replace('    ', '\t')  # 공백 4칸을 tab으로 변환

            if line == '':  # 전처리 후 빈 라인은 skip
                continue

            new_script.append(line)

        new_script = '\n'.join(new_script)  # 개행 문자로 합침
        # C++ 특정 문자열 패턴은 적용할 필요가 없으므로 제거합니다.
        # new_script = re.sub(r'/^(http?|https?):\/\/([a-z0-9-]+\.)+[a-z0-9]{2,4}.*$/', '', new_script)

    return new_script

# 사용 예시
# processed_script = preprocess_cpp_script('example.cpp')

In [3]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    print(f"Seed set as {seed}")

seed_everything(42)

Seed set as 42


In [4]:
code_folder = "../data/train_code"
problem_folders = os.listdir(code_folder) # directory에 있는 폴더 list를 반환

In [5]:
preprocess_scripts = []
problem_nums = []

# 300개 Sample code에 대한 전처리
for problem_folder in tqdm(problem_folders):
    scripts = os.listdir(os.path.join(code_folder, problem_folder)) # code/problem000/.py 파일
    problem_num = problem_folder # 문제 번호 폴더명
    for script in scripts:
        script_file = os.path.join(code_folder,problem_folder,script)
        preprocessed_script = preprocess_script(script_file)

        preprocess_scripts.append(preprocessed_script)
    # 번호 목록을 만들어서 전처리한 dataframe에 함께 넣어줌
    problem_nums.extend([problem_num]*len(scripts))

100%|██████████| 500/500 [08:00<00:00,  1.04it/s]


In [6]:
df_preprocessed = pd.DataFrame(data= {'code':preprocess_scripts, 'problem_num':problem_nums})
df_preprocessed = df_preprocessed[df_preprocessed['code'].str.len() > 10]


In [7]:
df_preprocessed

,code,problem_num
0,#include <bits/stdc++.h>\nusing namespace std;...,problem001
1,#include<bits/stdc++.h>\nusing namespace std;\...,problem001
2,#include <bits/stdc++.h>\nusing namespace std;...,problem001
3,#include <bits/stdc++.h>\nusing namespace std;...,problem001
4,#include <bits/stdc++.h>\n#define FAST ios_bas...,problem001
...,...,...
249995,#include <iostream>\nusing namespace std;\nint...,problem500
249996,#include <algorithm>\n#include <array>\n#inclu...,problem500
249997,#include <bits/stdc++.h>\nusing namespace std;...,problem500
249998,#include<iostream>\n#include<string>\nusing na...,problem500


In [8]:
# AutoTokenizer로 graphcodebert 사용하도록 설정
tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
tokenizer.truncation_side = 'left'
MAX_LEN = 512

tokens = []
for code in df_preprocessed['code']:
    tokens.append(tokenizer.tokenize(code, max_length=MAX_LEN, truncation=True))

df_preprocessed['tokens'] = tokens # Sample code를 Tokenization해서 tokens 컬럼에 추가
df_preprocessed['len'] = df_preprocessed['tokens'].apply(len) # tokens의 길이를 len 컬럼에 추가

In [10]:
df_preprocessed.to_csv('../data/train_graphbert.csv', index=False)

In [2]:
df_preprocessed = pd.read_csv("../data/train_graphbert.csv")

In [3]:
df_preprocessed = df_preprocessed.sample(frac=0.2, random_state=42)

In [4]:
# train과 validation data set 분리
train_df, valid_df, train_label, valid_label = train_test_split(
    df_preprocessed,
    df_preprocessed['problem_num'],
    random_state=42,
    test_size=0.1,
    stratify=df_preprocessed['problem_num']
)

train_df = train_df.reset_index(drop=True) # Reindexing
valid_df = valid_df.reset_index(drop=True)

In [5]:
len(train_df)

44819

In [6]:
codes = train_df['code'].to_list() # code 컬럼을 list로 변환 - codes는 code가 쭉 나열된 형태임
problems = train_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

In [7]:
total_positive_pairs = []
total_negative_pairs = []

In [8]:
for problem in tqdm(problems):
    # 각각의 문제에 대한 code를 골라 정답 코드로 저장, 아닌 문제는 other_codes로 저장
    # 이때 train_df에는 problem_num이 정렬된 상태가 아니기 때문에 index가 다를 수 있음
    solution_codes = train_df[train_df['problem_num'] == problem]['code'].to_list()
    other_codes = train_df[train_df['problem_num'] != problem]['code'].to_list()

    # positive_pairs 1000개 (총 300 * 1000 = 300,000개) 추출
    # negative_pairs 1000개 (총 300 * 1000 = 300,000개) 추출
    positive_pairs = list(combinations(solution_codes,2))
    random.shuffle(positive_pairs)
    positive_pairs = positive_pairs[:1000]
    random.shuffle(other_codes)
    other_codes = other_codes[:1000]

    negative_pairs = []
    for pos_codes, others in zip(positive_pairs, other_codes):
        negative_pairs.append((pos_codes[0], others))

    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)

100%|██████████| 500/500 [00:19<00:00, 25.26it/s]


In [16]:
# total_positive_pairs와 negative_pairs의 정답 코드를 묶어 code1로 지정
# total_positive_pairs와 negative_pairs의 비교 대상 코드를 묶어 code2로 지정
# 해당 코드에 맞는 label 설정
code1 = [code[0] for code in total_positive_pairs] + [code[0] for code in total_negative_pairs]
code2 = [code[1] for code in total_positive_pairs] + [code[1] for code in total_negative_pairs]
label = [1]*len(total_positive_pairs) + [0]*len(total_negative_pairs)

# DataFrame으로 선언
train_data = pd.DataFrame(data={'code1':code1, 'code2':code2, 'similar':label})
train_data = train_data.sample(frac=1).reset_index(drop=True) # frac: 추출할 표본 비율
train_data.to_csv('../data/train_data_lv1.csv',index=False)

In [15]:
train_data.head()

,code1,code2,similar
0,#include <bits/stdc++.h>\nusing namespace std;...,"#include<stdio.h>\nint main()\n{\n int w,h,x,y...",0
1,#include<bits/stdc++.h>\n#define ll long long\...,#include <cmath>\n#include <iostream>\n#includ...,0
2,"#include <bits/stdc++.h>\n#define rep(i,a,b) f...","#include <bits/stdc++.h>\n#define range(i, a, ...",0
3,#include <bits/stdc++.h>\n#define ALL(a) (a)....,#include <stdio.h>\n#include <iostream>\n#incl...,0
4,#include <bits/stdc++.h>\n#define ALL(a) (a).b...,#include <iostream>\n#include <string.h>\n#inc...,0


In [10]:
codes = valid_df['code'].to_list() # code 컬럼을 list로 변환 - codes는 code가 쭉 나열된 형태임
problems = valid_df['problem_num'].unique().tolist() # 문제 번호를 중복을 제외하고 list로 변환
problems.sort()

In [11]:
total_positive_pairs = []
total_negative_pairs = []

In [12]:
for problem in tqdm(problems):
    # 각각의 문제에 대한 code를 골라 정답 코드로 저장, 아닌 문제는 other_codes로 저장
    # 이때 train_df에는 problem_num이 정렬된 상태가 아니기 때문에 index가 다를 수 있음
    solution_codes = valid_df[valid_df['problem_num'] == problem]['code'].to_list()
    other_codes = valid_df[valid_df['problem_num'] != problem]['code'].to_list()

    # positive_pairs 100개 (총 300 * 100 = 30,000개) 추출
    # negative_pairs 100개 (총 300 * 100 = 30,000개) 추출
    positive_pairs = list(combinations(solution_codes,2))
    random.shuffle(positive_pairs)
    positive_pairs = positive_pairs[:100]
    random.shuffle(other_codes)
    other_codes = other_codes[:100]

    negative_pairs = []
    for pos_codes, others in zip(positive_pairs, other_codes):
        negative_pairs.append((pos_codes[0], others))

    total_positive_pairs.extend(positive_pairs)
    total_negative_pairs.extend(negative_pairs)

100%|██████████| 500/500 [00:02<00:00, 213.07it/s]


In [17]:
# total_positive_pairs와 negative_pairs의 정답 코드를 묶어 code1로 지정
# total_positive_pairs와 negative_pairs의 비교 대상 코드를 묶어 code2로 지정
# 해당 코드에 맞는 label 설정
code1 = [code[0] for code in total_positive_pairs] + [code[0] for code in total_negative_pairs]
code2 = [code[1] for code in total_positive_pairs] + [code[1] for code in total_negative_pairs]
label = [1]*len(total_positive_pairs) + [0]*len(total_negative_pairs)

# DataFrame으로 선언
valid_data = pd.DataFrame(data={'code1':code1, 'code2':code2, 'similar':label})
valid_data = valid_data.sample(frac=1).reset_index(drop=True) # frac: 추출할 표본 비율
valid_data.to_csv('../data/valid_data_lv1.csv',index=False)